<a href="https://colab.research.google.com/github/KhanShaheb34/BD-ProtidinScraper/blob/master/ScrapeBDProtidin.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [26]:
import bs4
import requests
import os
import pandas as pd
import re

In [51]:
def getCategoriesFromDate(date):
  url = f"https://www.bd-pratidin.com/archive/{date}"
  page = requests.get(url)
  soup = bs4.BeautifulSoup(page.content, "html.parser").find("div", {"class": "container-left-area printversion col-md-9"})
  categories = set()

  for a in soup.findAll("a", {"href": re.compile(r"^[a-z-]+\/\d+\/\d+\/\d+")}):
    categories.add(a["href"].split("/")[0])

  return list(categories)

In [60]:
def getNewsFromCategoryAndDate(category, date):
  url = f"https://www.bd-pratidin.com/{category}/{date}"
  page = requests.get(url)
  soup = bs4.BeautifulSoup(page.content, "html.parser").find("div", {"class": "container-left-area col-md-9"})
  newsLinks = []

  for a in soup.findAll("a", {"href": re.compile(r"^[a-z-]+\/\d+\/\d+\/\d+\/\d+$")}):
    newsLinks.append(a["href"])

  return newsLinks

In [86]:
def getNewsFromLink(link):
  url = f"https://www.bd-pratidin.com/{link}"
  page = requests.get(url)
  soup = bs4.BeautifulSoup(page.content, "html.parser")

  title = soup.find("h1", {"class":"post-title"}).text.strip()
  description = soup.find("meta", {"property":"og:description"})["content"].strip()
  category = link.split("/")[0]
  id = int(link.split("/")[4])
  date = "/".join(link.split("/")[1:4])

  articleSoup = soup.find("article")
  article = ""
  for p in articleSoup.findAll("p"):
    article += p.text

  return {"id": id, "title": title, "description": description, "category": category, "date": date, "article": article}

In [87]:
date = "2020/07/07"                       # Change this date to get data from any date.
categories = getCategoriesFromDate(date)
newsLinks = []

for category in categories:
  newsLinks += getNewsFromCategoryAndDate(category, date)

news = []
for link in newsLinks:
  news.append(getNewsFromLink(link))

newsDataframe = pd.DataFrame(news)
newsDataframe

,id,title,description,category,date,article
0,545955,আজকের ভাগ্যচক্র,আজকের এই দিনে জন্মগ্রহণ করায় পাশ্চাত্যমতে আপনি...,horoscope,2020/07/07,আজকের এই দিনে জন্মগ্রহণ করায় পাশ্চাত্যমতে আপন...
1,545977,কর্পোরেট কর্নার,দেশীয় স্বর্ণ শিল্পের বিকাশ ও রপ্তানি বাণিজ্য স...,money-market-business,2020/07/07,দেশীয় স্বর্ণ শিল্পের বিকাশ ও রপ্তানি বাণিজ্য ...
2,546001,হঠাৎ প্রিয় মাঠে মুশফিক,মিরপুর বরাবরই প্রিয় মাঠ মুশফিকের। টেস্ট ক্রিকে...,sport-news,2020/07/07,মিরপুর বরাবরই প্রিয় মাঠ মুশফিকের। টেস্ট ক্রিক...
3,546002,অবশেষে বার্সার জয়,স্প্যানিশ লা লিগায় টানা দুই ম্যাচে পয়েন্ট হারি...,sport-news,2020/07/07,স্প্যানিশ লা লিগায় টানা দুই ম্যাচে পয়েন্ট হার...
4,546003,অনুদান না অপমান?,কেউ রিকশা চালাচ্ছেন। কেউবা ভ্যানে ফল ও সবজি বি...,sport-news,2020/07/07,কেউ রিকশা চালাচ্ছেন। কেউবা ভ্যানে ফল ও সবজি ব...
...,...,...,...,...,...,...
128,546078,সীমান্তের সেনা প্রত্যাহারে ভারত-চীন ঐকমত্য,ভারত-চীন সীমান্ত সংঘর্ষ বন্ধে দুই দেশের মধ্যে ...,last-page,2020/07/07,ভারত-চীন সীমান্ত সংঘর্ষ বন্ধে দুই দেশের মধ্যে...
129,546079,হজে এবার কালো পাথরে চুমু দেওয়া বন্ধ,করোনাভাইরাস মহামারীর কারণে এবার সীমিত পরিসরে হ...,last-page,2020/07/07,করোনাভাইরাস মহামারীর কারণে এবার সীমিত পরিসরে ...
130,546080,প্রবাসীদের ইকামা ও ভিসার মেয়াদ ৩ মাস বাড়াল সৌদি,সৌদি প্রবাসীদের মধ্যে যাদের বসবাসের অনুমতির (ই...,last-page,2020/07/07,সৌদি প্রবাসীদের মধ্যে যাদের বসবাসের অনুমতির (...
131,546081,পুলিশি নির্যাতনে কিডনি নষ্টের বিচার বিভাগীয় তদ...,যশোরে পুলিশের হাতে নির্মম নির্যাতনের কারণে ইমর...,last-page,2020/07/07,যশোরে পুলিশের হাতে নির্মম নির্যাতনের কারণে ইম...
